Daily Challenge: Build a Retrieval Augmented Generation (RAG) System

implémenter RAG en utilisant Langchain et Hugging Face

1. Set up your environment:

In [4]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
! pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install -U langchain-community

In [42]:
!pip install transformers langchain torch

In [6]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

2. Load the dataset:

In [7]:
# Définition du dataset et de la colonne contenant le texte:
dataset_name = "databricks/databricks-dolly-15k"
# La colonne contenant le texte
page_content_column = "context"

In [8]:
# Chargement des données
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)
data = loader.load()
print(data[:2]) # Optional: Vérification : Afficher les 2 premiers documents
# code charge le dataset depuis Hugging Face et affiche les deux premiers documents pour vérifier que tout fonctionne bien.

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.20k [00:00<?, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

[Document(metadata={'instruction': 'When did Virgin Australia start operating?', 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}, page_content='"Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia\'s domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney."'), Document(metadata={'instruction': 'Which is a species of fish? Tope or Rope', 'response': 'Tope', 'category': 'classification'}, page_content='""')]


3. Split the documents:

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

In [10]:
docs = text_splitter.split_documents(data)
print(docs[0]) # Optional: Print the first document chunk

page_content='"Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney."' metadata={'instruction': 'When did Virgin Australia start operating?', 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}


4. Embed the text:

In [11]:
# Define the model path, model configurations, and encoding options(Définition du modèle et des paramètres):
modelPath = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device':'cpu'} #Indique que le modèle sera exécuté sur le CPU
encode_kwargs = {'normalize_embeddings': False} # Indique qu'on ne normalise pas les embeddings (optionnel).

In [12]:
# Initialize HuggingFaceEmbeddings (Initialisation du modèle d'embeddings):
embeddings = HuggingFaceEmbeddings(
model_name=modelPath,
model_kwargs=model_kwargs,
encode_kwargs=encode_kwargs
)

<ipython-input-12-84b77cf64d97>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
# Test embedding creation (Tester la génération d’un embedding pour une phrase):
text = "This is a test document."
query_result = embeddings.embed_query(text) #Convertit la phrase en vecteur.
print(query_result[:3])

[-0.038338541984558105, 0.1234646886587143, -0.028642907738685608]


5. Create a vector store:

In [14]:
from langchain.vectorstores import FAISS

En pratique, on doit toujours extraire le texte et générer les embeddings avant d’enregistrer quoi que ce soit dans FAISS.

In [15]:
docs_texts = [doc.page_content for doc in data]  # Liste des textes des documents/FAISS ne comprend pas les objets Document, il a besoin de texte pur/extrait le texte de chaque document

In [16]:
docs_embeddings = embeddings.embed_documents(docs_texts)

In [17]:
# Create a FAISS vector store from the document chunks and embeddings:
db = FAISS.from_documents(docs, embeddings)

Effectuer une Recherche de Documents dans FAISS

In [27]:
from langchain.vectorstores import FAISS

In [28]:
def retrieve_documents(query, top_k=3): #Recherche les k documents les plus proches dans FAISS.
    """
    Recherche les documents les plus pertinents pour une requête donnée.
    Args:
        query (str): La question posée par l'utilisateur.
        top_k (int): Nombre de documents à récupérer.
    Returns:
        List: Liste des documents les plus pertinents.
    """
    # Convertir la question en embedding
    query_embedding = embeddings.embed_query(query)

    # Rechercher les top_k documents les plus proches dans FAISS
    retrieved_docs = vectorstore.similarity_search_by_vector(query_embedding, k=top_k)

    return retrieved_docs

6. Prepare the LLM model:

In [43]:
# 1 importe les bibli necessaires
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from langchain import HuggingFacePipeline

--> Create a question-answering pipeline:

In [44]:
# 2 Définir le nom du modèle
model_name = "Intel/dynamic_tinybert"


In [45]:
# 3️ Charger le tokenizer et le modèle de question-réponse
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

Invalid model-index. Not loading eval results into CardData.


In [46]:
# 4️ Créer un pipeline de question-réponse
qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    return_tensors='pt'  # Utilisation de PyTorch pour accélérer les calculs
)

Device set to use cpu


-->Create a Langchain pipeline wrapper:

In [47]:
# 5️ Intégrer le modèle dans LangChain en tant que `llm`
llm = HuggingFacePipeline(
    pipeline=qa_pipeline,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)

<ipython-input-47-9c266437325e>:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(


In [54]:
# 6️ Vérification
print("\n Modèle `Intel/dynamic_tinybert` chargé et intégré avec succès dans LangChain !")
# Ce code prépare le modèle LLM et l’intègre sous la variable llm, qui pourra être utilisée pour générer des réponses dans RetrievalQA.


 Modèle `Intel/dynamic_tinybert` chargé et intégré avec succès dans LangChain !


7. Build the Retrieval QA Chain:

--> Create a retriever from your FAISS database:

In [49]:
retriever = db.as_retriever(search_kwargs={"k": 4})  # `k` est le nombre de documents récupérés

--> Build the RetrievalQA chain:

In [50]:
qa = RetrievalQA.from_chain_type(
      llm=llm, # Modèle LLM chargé précédemment
      chain_type="refine", # Mode de réponse : amélioration progressive de la réponse
      retriever=retriever, # Utilisation de FAISS pour récupérer les documents
      return_source_documents=False # Ne pas afficher les documents sources (optionnel)
      )

8. Test your RAG system with a question :

-->Define your question:

In [51]:
# 1️ question utilisateur
query = "Qu'est-ce que LangChain ?"

-->Run the QA chain and print the result:

In [56]:
# 2️ interroge `RetrievalQA`
response = qa.run(query)

/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


ValueError: Context information is below. 
------------
le cadre de la tentative de contre-r\u00e9volution de la guerre de Vend\u00e9e, au cours de laquelle plusieurs centaines de milliers de personnes trouv\u00e8rent la mort, lors des insurrections f\u00e9d\u00e9ralistes ou dans le cadre de luttes entre factions r\u00e9volutionnaires rivales, qui ont abouti \u00e0 la mort successive des principales figures r\u00e9volutionnaires."
------------
Given the context information and not prior knowledge, answer the question: Qu'est-ce que LangChain ?
 argument needs to be of type (SquadExample, dict)

In [53]:
# 3️⃣ Afficher la réponse générée
print("\n Réponse générée :" {response})


SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-53-0abca3d3cadb>, line 2)

7.Pipeline RAG manuel

--> Intégration Complète de RAG (Recherche + Réponse)\n Plan du Pipeline Complet: 1️ L’utilisateur pose une question.
2️ On transforme la question en embedding.
3️ On recherche dans FAISS les documents les plus pertinents.
4️ On passe ces documents au modèle de question-réponse (Intel/dynamic_tinybert).
5️\ Le modèle génère une réponse en s’appuyant sur ces documents.


In [25]:
# 1️⃣ Fonction pour exécuter tout le pipeline RAG
def rag_pipeline(query, top_k=3): # rag_pipeline(), exécute tout le processus en une seule commande.
    """
    Pipeline complet de RAG : Recherche de documents + Génération de réponse.
    Args:
        query (str): Question posée par l'utilisateur.
        top_k (int): Nombre de documents à récupérer.
    Returns:
        str: Réponse générée.
    """
    print("\n Recherche des documents les plus pertinents...")
    retrieved_docs = retrieve_documents(query, top_k=top_k)

    # Afficher les documents récupérés
    print("\n Documents récupérés :")
    for i, doc in enumerate(retrieved_docs):
        print(f"\n Document {i+1} :\n{doc.page_content}")

    # Construire le contexte à partir des documents récupérés
    context = " ".join([doc.page_content for doc in retrieved_docs])

    print("\n Génération de la réponse avec le modèle de question-réponse...")
    qa_input = {
        "question": query,
        "context": context
    }

    # Générer une réponse en utilisant le modèle QA
    answer = qa_pipeline(qa_input)

    return answer["answer"]

In [26]:
# 2️⃣ Tester le pipeline avec une question
query = "Qu'est-ce que LangChain ?"
response = rag_pipeline(query, top_k=3)


 Recherche des documents les plus pertinents...


NameError: name 'vectorstore' is not defined

In [ ]:
# 3️⃣ Afficher la réponse finale
print("\n✅ Réponse finale :")
print(response)
# -->Ce code exécute toutes les étapes de RAG en une seule fois !